# Data Preprocessing using RAPIDS and Training XGBoost for Fraud Detection

<img src="./images/rapids.png" alt="rapids" width="400" align="center"/>

In this notebook we will walk through using [RAPIDS](https://rapids.ai/about.html) for GPU-accelerated data preprocessing and training of XGBoost model for Fraud Detection use-case. In the [second notebook](2_triton_xgb_fil_ensemble.ipynb) we will show how to deploy the trained XGBoost model in Triton on SageMaker. The RAPIDS suite of open source software libraries and APIs gives you the ability to execute end-to-end data science and analytics pipelines entirely on GPUs.



**Note:** Since the primary goal of this example is to get a trained XGBoost model to illustrate deployment of Tree-based ML models on Triton in SageMaker we don't perform any in-depth feature engineering or hyperparameter optimization. Although RAPIDS on SageMaker is excellent for [running cost-effective HPO in minimal amount of time](https://aws.amazon.com/blogs/machine-learning/rapids-and-amazon-sagemaker-scale-up-and-scale-out-to-tackle-ml-challenges/) to get to the best accuracy model configuration. 

## To Run This Notebook Please Select RAPIDS 2106 Kernel from the Kernel Dropdown menu

This notebook was tested with the `rapids-2106` kernel on an Amazon SageMaker notebook instance of type `g4dn`.

## Get Data

For this example, we use the Tabformer [synthetic credit card transactions dataset](https://arxiv.org/abs/1910.03033) from IBM available on [Kaggle](https://www.kaggle.com/datasets/ealtman2019/credit-card-transactions). You can either directly download the dataset from this [Kaggle link](https://www.kaggle.com/datasets/ealtman2019/credit-card-transactions) and then upload it to your SageMaker notebook instance. Or you may fetch the data from Kaggle command line client using the following commands.


### Set up Kaggle API

First we install the Kaggle CLI.

In [ ]:
!pip install -q kaggle

Then we enable the Kaggle API. This assumes you have an account on Kaggle. It's free and only takes a minute. Once you have that, follow [instructions here](https://github.com/Kaggle/kaggle-api#api-credentials) to retrieve your kaggle.json file and upload it to SageMaker through JupyterLab upload interface. Then run the following cells.

In [ ]:
!mkdir /home/ec2-user/.kaggle
!mv kaggle.json /home/ec2-user/.kaggle/
!chmod 600 /home/ec2-user/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ealtman2019/credit-card-transactions

In [ ]:
!unzip -u credit-card-transactions.zip

## Check on our GPU

Next, let's check the GPU resources we have by using the terminal command `nvidia-smi`.

In [1]:
!nvidia-smi
!nvidia-smi -L

Wed Jul 20 17:53:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:41:00.0  On |                  Off |
| 30%   47C    P8    36W / 300W |   1166MiB / 49140MiB |     25%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Awesome, we have powerful NVIDIA GPU at our disposal. Let's get started with using it for Data Preprocessing.

## Data Preprocessing

In [2]:
import cudf
import cuml
import numpy as np
import pickle
import os

We read in the data and begin our data preprocessing.

In [3]:
data_path = './'
data_csv = 'credit_card_transactions-ibm_v2.csv'
full_data = cudf.read_csv(os.path.join(data_path, data_csv))
full_data.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,<NA>,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,<NA>,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,<NA>,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,<NA>,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,<NA>,No


Each row here is a credit card transaction with its attributes like time and amount of transaction along with merchant attributes like Name, City, State, Zipcode and Merchant Category Code (MCC) and finally whether the transaction was fraudulent or legitimate (`Is Fraud?`). 

**Note:** `Merchant Name` is hashed so that's why we see integers instead of strings.

The full dataset has about 24 million rows but in this example we use random subset of about ~5 million transactions.

In [4]:
SEED = 42
data = full_data.sample(frac=0.2, random_state=SEED)
data = data.reset_index(drop=True)
print(data.shape)

(4877380, 15)


We convert some categorical features to dtype object.

In [5]:
data['Zip'] = data['Zip'].astype('object')
data['MCC'] = data['MCC'].astype('object')
data["Merchant Name"] = data["Merchant Name"].astype("object")

### Encode labels


Next we perform encoding on our binary labels `Is Fraud?` which indicate whether a transaction is fraudulent or not. After encoding `1` will denote fraud and `0` will denote legitimate transaction.

In [6]:
y = data['Is Fraud?']
data.drop(columns=['Is Fraud?'], inplace=True)
y = (y == "Yes").astype(int)

### Save subset for inference

We will also save a small subset of the data to submit Triton inference requests for later on in the [second notebook](2_triton_xgb_fil_ensemble.ipynb).

In [7]:
data_infer = data.iloc[510:516]
data_infer.to_csv('data_infer.csv', index=False)

### Handle Missing Values

Next let's handle the missing values in our data.

In [8]:
data.isna().sum()/len(data) * 100

User               0.000000
Card               0.000000
Year               0.000000
Month              0.000000
Day                0.000000
Time               0.000000
Amount             0.000000
Use Chip           0.000000
Merchant Name      0.000000
Merchant City      0.000000
Merchant State    11.143544
Zip               11.791884
MCC                0.000000
Errors?           98.402626
dtype: float64

We have some missing values in `Merchant State` and `Zip` columns. Turns out these correspond to Online transactions so we will set those missing values to `ONLINE`.

In [9]:
data.loc[data["Merchant City"]=="ONLINE", "Merchant State"] = "ONLINE" 
data.loc[data["Merchant City"]=="ONLINE", "Zip"] = "ONLINE" 

We also have some foreign transactions where `Merchant City` and `Merchant State` is a foreign city and country and the Zipcode is missing. For those transactions we will set the Zipcode to `FOREIGN`.

In [10]:
us_states_plus_online = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', 'ONLINE']

# set zip of all transactions that are not in US States or Online to Foreign
data.loc[~data["Merchant State"].isin(us_states_plus_online), "Zip"] = "FOREIGN"

The `Errors?` column indicates whether or not the transaction had any errors like Incorrect Pin associated with it. We make this a boolean indicator feature.

In [11]:
data['Errors?'] = data['Errors?'].notna()

In [12]:
data.isna().sum()/len(data) * 100

User              0.0
Card              0.0
Year              0.0
Month             0.0
Day               0.0
Time              0.0
Amount            0.0
Use Chip          0.0
Merchant Name     0.0
Merchant City     0.0
Merchant State    0.0
Zip               0.0
MCC               0.0
Errors?           0.0
dtype: float64

So now we have handled all the missing values in our data.

### Handle Amount and Time

Next, for the `Amount` column we remove the dollar symbol prefix and for `Time` column we extract out the Hour and Minute.

In [14]:
data['Amount'] = data['Amount'].str.slice(1)
data['Hour'] = data['Time'].str.slice(stop=2)
data['Minute'] = data['Time'].str.slice(start=3)
data.drop(columns=['Time'], inplace=True)

###  Train-Test Split

Before doing any further preprocessing let's perform the train-test split. Here we use 70-30 train-test split.

In [15]:
from cuml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=SEED, stratify=y)

In [16]:
# Free up some room on the GPU by explicitly deleting dataframes
import gc
del data
del y
gc.collect()

763

### Encoding Categorical Columns

Next, we handle categorical columns in our dataset by performing [label encoding](https://docs.rapids.ai/api/cuml/stable/api.html?highlight=label%20encoder#feature-and-label-encoding-single-gpu) on them which converting categorical values into numerical values. For some of these columns we have some unseen values which are present in test data but not train data. We handle those values by setting them to `UNKNOWN` before doing the label encoding so that at test time we have an encoding for these unseen values.

We also serialize the encodings for all categorical columns so that we can later use them for doing data preprocessing at inference time in the [second notebook](2_triton_xgb_fil_ensemble.ipynb).

In [17]:
from cuml.preprocessing import LabelEncoder
categorial_columns = ['Zip', 'MCC', 'Merchant Name', 'Use Chip', 'Merchant City', 'Merchant State']
encoders = {}

# handle unknown values present in test data but not in training data
for col in categorial_columns:
    # convert cudf series to numpy array with .values_host
    unique_values = X_train[col].unique().values_host
    X_test.loc[~X_test[col].isin(unique_values), col] = 'UNKNOWN'
    unique_values = np.append(unique_values, ['UNKNOWN'])
    # convert numpy array to cudf series
    unique_values = cudf.Series(unique_values)
    le = LabelEncoder().fit(unique_values)
    X_train[col] = le.transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
    encoders[col] = le.classes_.values_host

with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f)

In [18]:
# convert all dtypes to fp32 for xgboost training
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

Let's look at our preprocessed data.

In [19]:
X_train.head()

,User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Hour,Minute
3611536,631.0,0.0,2011.0,10.0,14.0,2.610000,2.0,14457.0,7962.0,67.0,22525.0,78.0,0.0,8.0,33.0
834552,1999.0,1.0,2018.0,1.0,25.0,47.410000,1.0,15639.0,7466.0,140.0,23344.0,35.0,0.0,7.0,34.0
3068385,1691.0,1.0,2008.0,8.0,16.0,42.080002,1.0,3451.0,7466.0,140.0,23344.0,41.0,0.0,8.0,27.0
2481906,1038.0,1.0,2008.0,5.0,24.0,8.920000,2.0,38748.0,763.0,2.0,7409.0,71.0,0.0,8.0,59.0
519653,1881.0,1.0,2013.0,10.0,6.0,11.360000,2.0,28581.0,2018.0,139.0,17169.0,74.0,0.0,19.0,21.0


## Train XGBoost

Now we train the XGBoost fraud detection model on our GPU. This will take about 2-3 minutes on `g4dn.xlarge` instance.

In [ ]:
import xgboost as xgb
import time

dtrain = xgb.DMatrix(
        X_train,
        y_train)

dtest = xgb.DMatrix(
        X_test,
        y_test)

max_depth = 8
num_trees = 2000
xgb_params = {
    'max_depth':          max_depth,
    'tree_method':       'gpu_hist',
    'objective':         'binary:logistic',
    'eval_metric':       'aucpr',
    'predictor':         'gpu_predictor',
}
model = xgb.train(params=xgb_params, 
                       dtrain=dtrain, 
                       num_boost_round=num_trees)

We quickly evaluate our trained model's predictions on the test set using F1-score.

In [ ]:
from sklearn.metrics import f1_score

y_score = model.predict(dtest)
threshold = 0.5
y_pred = (y_score >= 0.5).astype(int)
y_true = y_test.values_host
f1 = f1_score(y_true, y_pred)
print(f'Test F1-Score: {f1: 0.4f}')

We can do further Hyperparameter tuning/Feature Engineering to improve the model accuracy but since the primary goal of this example is to walkthrough deployment of decision tree-based ML models like XGBoost on Triton in SageMaker we save our trained model and move on to the [second notebook](2_triton_xgb_fil_ensemble.ipynb).

### Save Trained Model

In [ ]:
model_path = "./xgboost.json"
model.save_model(model_path)

## Next Step

Please open the [second notebook](2_triton_xgb_fil_ensemble.ipynb) to learn how to deploy this XGBoost model and other similar decision tree-based ML models on Triton in SageMaker.